# Self teaching auto completion engine

## Part 1 - Predicting a single token based on the previous word

In the first part of this notebook we will explore whether it is possible to train a neural network to predict one word of source code given another word, that is, toss one word at it and make it guess which one comes next. We will do this with Python source code as it is widely used and easy to learn.

### Why do we do this

Auto completion is such a nice feature to have in a text editor. Basically every at least moderately decent text editor is capable of doing completion based on words that you have entered before. [Vim](http://www.vim.org/) for example can even complete whole lines for you (C-X C-L).

The next level of completion somewhat context based, as in most IDEs. You can enter the name of a Java object, followed by a dot and the IDE shows you a list of available methods.

And then there are [snippets](https://github.com/SirVer/ultisnips). You just type a certain keyword like ```class``` and it gets expanded into a whole construct, containing all the placeholders and boilerplate that make sure that your code runs smoothly and is well documented.

The last two is basically what you want and need as a programmer to avoid retyping boilerplate over and over again and to remind yourself from time to time to structure and document your code well. But these two rely on someone else doing the dirty work for you. Someone else has to implement rules and mechanisms that allow you to use these kinds of completion. Plus, all this work needs to be redone for other programming languages that may arise.

### Fetching the data

First we will need to dowload a ton of Python code. We chose the ActiveState Git repository, as it contains a hell lot of code that is mostly high quality.

In [1]:
! ([ -d code ] && echo "Code directory already present") || git clone https://github.com/ActiveState/code.git

Cloning into 'code'...
remote: Counting objects: 68352, done.
remote: Total 68352 (delta 0), reused 0 (delta 0), pack-reused 68352
Receiving objects: 100% (68352/68352), 12.05 MiB | 0 bytes/s, done.
Resolving deltas: 100% (24509/24509), done.
Checking connectivity... done.


### Inspecting the data

Before we begin with our actual work, it is always beneficial to take a look at the data first, so you now what you will be dealing with. This is especially important in our case as we are working with a programming language, which looks way different than natural language.

#### Gather all source files

We will traverse the code directory and absorb every file that ends with .py, which should be a good enough indicator that this is indeed python source code.

In [2]:
import os

python_files = []
for root, dirs, files in os.walk('./code'):
    for file in files:
        if file.endswith('.py'):
            python_files += [ root + "/" + file ]

num_files = len(python_files)
print(str(num_files) + " files loaded")

4597 files loaded


That wasn't so hard, right? Now that we have a list of all relevant files we will read all of them in and save the whole text as one giant string.

In [3]:
blob = ""
for file in python_files:
    with open(file) as f:
        blob += f.read()

all_words = blob.split()

print("Read all files")

Read all files


### Cleaning the data

We will now use the Counter class to generate a list of all words and their corresponding frequency in the whole corpus. Python will do this more or less automatically for us, so we don't have to write too much source code here. After that, let's look at some of the most frequent tokens.

In [4]:
from collections import Counter

word_freqs = Counter(all_words)

for w in word_freqs.most_common(20):
    print(w)

('=', 91988)
('#', 43096)
('if', 33328)
('def', 30766)
('the', 29026)
('in', 23738)
('return', 20957)
('for', 20362)
('to', 14870)
('a', 14759)
('of', 12784)
('==', 12583)
('print', 12504)
('and', 12445)
('is', 12422)
('+', 12064)
('import', 10869)
('"""', 8902)
('not', 8452)
('1', 7927)


Since we are dealing with a programming language, it is no big surprise that something as trivial as an equal sign is on top of the list. But we can also see that the corpus contains a lot of 'normal' english words. This may pose a problem later because we don't want to help the person in front of the screen to write better prose, but nice and clean source code. On the other hand, python contains a lot of english words like _for_ or _if_ as reserved key words. Of cource we wouldn't want to kick these out of our data.

Fortunately, Python offers a way to get out of such situation. Take a look at this:

In [5]:
import keyword

print(keyword.iskeyword('def'))
print(keyword.kwlist[:6])

True
['False', 'None', 'True', 'and', 'as', 'assert']


How nice! We will use this a little bit later on. For now, let's focus on cleaning up the data by removing anything that isn't that important (think of comments for example).

In [6]:
# Step 1 - remove all comments, including inline ones, while preserving the source code in front of them

import re

comments = re.compile(r'#.*')

cleaned_text = comments.sub('', blob)

Great! Now we will the special characters dot, comma and end-of-line a special treatment so they stand out a little bit more.

In [7]:
# Step 2 - Pad dots and commas with whitespace, so things like "a," and "a ," are both recognized
# as an 'a' followed by a comma

commas = re.compile(r',')
cleaned_text = commas.sub(' , ', cleaned_text)

dots = re.compile(r'\.')
cleaned_text = dots.sub(' . ', cleaned_text)

# Line breaks can also cause trouble if the next line is not indented

cr = re.compile(r'\n')
cleaned_text = cr.sub(' <EOL> ', cleaned_text)

And now it's time to seperate brackets and operators from things like variable names by putting some whitespace between them.

In [8]:
# Now just add some spaces to all kinds of brackets
cleaned_text = re.sub(r'\(', ' ( ', cleaned_text)
cleaned_text = re.sub(r'\[', ' [ ', cleaned_text)
cleaned_text = re.sub(r'\{', ' { ', cleaned_text)
cleaned_text = re.sub(r'\)', ' ) ', cleaned_text)
cleaned_text = re.sub(r'\]', ' ] ', cleaned_text)
cleaned_text = re.sub(r'\}', ' } ', cleaned_text)

# Equal signs are also special. We don't want something like "a=b" clobber our dictionary,
# so we add spaces. Then we undo this transformation on the comparison operator "==" and some special
# assignments
cleaned_text = re.sub(r'=', ' = ', cleaned_text)
cleaned_text = re.sub(r'=  =', '==', cleaned_text)
cleaned_text = re.sub(r'\+ =', '\+=', cleaned_text)
cleaned_text = re.sub(r'- =', '-=', cleaned_text)
cleaned_text = re.sub(r'\* =', '\*=', cleaned_text)
cleaned_text = re.sub(r'/ =', '/=', cleaned_text)

# Let's do the same with all the operators
cleaned_text = re.sub(r'\+', ' \+ ', cleaned_text)
cleaned_text = re.sub(r'-', ' - ', cleaned_text)
cleaned_text = re.sub(r'\*', ' \* ', cleaned_text)
cleaned_text = re.sub(r'/', ' / ', cleaned_text)

cleaned_text = re.sub(r'\+ =', '\+=', cleaned_text)
cleaned_text = re.sub(r'- =', '-=', cleaned_text)
cleaned_text = re.sub(r'\* =', '\*=', cleaned_text)
cleaned_text = re.sub(r'/ =', '/=', cleaned_text)


print("Done cleaning!")

Done cleaning!


### Transforming the data

The next step is to turn the words into numbers. Therefore we will create two dictionaries, one that translates words into numbers and another one that translates numbers back into words (for performance reasons).

In [9]:
def create_dicts(word_list):
    
    # The cleaned word list may contain tokens that only consist of whitespace,
    # or that are padded with whitespace.
    # Here we delete all whitespace in a token and drop it if nothing remains
    cleaned_word_list = [ w.strip() for w in word_list if w.strip() != '' ]
    
    # Now we add a token that will later represent everything that we think
    # is the name of a variable, constant, or any other kind of identifier
    cleaned_word_list += ['<ID>']
    
    # Now we make all tokens unique. This is necessary to ensure that the values
    # are uninterrupted
    cleaned_word_list = set(cleaned_word_list)
    
    # Now we will create the tuples that consist of the future
    # key value pairs. We have to cast this into a list because
    # the enumerate object won't return anything after its first
    # usage
    word_idx_pairs = list(enumerate(cleaned_word_list))

    # Dictionary comprehension. This is now trivial thanks to our
    # work before
    w2n_dict = { w: i for i, w in word_idx_pairs}
    n2w_dict = { i: w for i, w in word_idx_pairs}

    return w2n_dict, n2w_dict

all_words = cleaned_text.split(" ")
w2n_dict, n2w_dict = create_dicts(all_words)

Let's see if it works:

In [10]:
n2w_dict[w2n_dict['return']]

'return'

Looks like the lookup is working in both directions. Let's see how big our dictionary has gotten:

In [11]:
print(len(w2n_dict))

151441


This is still a bit much. Let's see what's inside:

In [12]:
print(list(w2n_dict.keys())[:50])

['8pNrLmSzWonUm', 'scanlog', '"defining', 'imginfo', '"Hahha', "'__name__':", 'Thus:', 'Sp3NvM7xI6axP', 'globalns', 'cZ5aYGqPHJBI8OClBzQ', 'xv"', '$%d', 'html_month', 'a', 'NoneSuch', 'xOeOdVibtD9jNm', 'mP6OsnyO', 'WHATSOEVER', 'renderMissingNames', 'P5ekC', '<msg>:', 'etext92', 'norman', "com:587';", '<dir1>', 'disabled:', 'r":', "8'>", "mp3':", 'XMLTreeBuilder:', 'RELATIVE', 'disableedit', 'TemporaryFile', '"output"', 'lExpandFile', 'n_rows\\n"', 'putPage', 'wordBoundaries', "search'", '"FiaQ5', 'EN"', '_code', 'ModuleReturn', 'makeCode', 'CodeWrapper:', 'longJob', '"PgbSN8O3E', 'resumes', 'j36cPJ', '?\\n?"']


Yikes, what a mess this still is!

One thing that is commonly used in NLP (natural language processing) is cutting of the lower end of your word distribution. That is, take all the words that occur less then a certain threshold and just throw them overboard. Let's do this!

In [13]:
# The word count should be at least a certain fraction of the total
# number of documents
min_fraction = 0.2
min_count = num_files * min_fraction

# Cut off words that are simply too rare
all_words = cleaned_text.split(" ")
c = Counter(all_words)

reduced_words = [ w for w in all_words if c[w] >= min_count ]

reduced_w2n_dict, reduced_n2w_dict = create_dicts(reduced_words)

print("Before: " + str(len(w2n_dict)))
print("After:  " + str(len(reduced_w2n_dict)))

Before: 151441
After:  214


### Making the data ready for training

Ok so now that we have greatly reduced our dictionary size, we still have one problem to solve. The words that were filtered out are actually still inside the text and appear alongside the words that we are interested in. So when creating the input label pairs, what should we do?

In our case, we will just silently ignore these cases and return nothing. So let's get to it.

First we will transform every line into a series of IDs, masking all words that we don't care about with a -1. We will then say, that all these '-1's are probably some sort of identifier for a constant or variable and will call it just ```<ID>``` later.

In [14]:
def sentence_to_num(sent, ldict):
    transformed = []
    for w in sent.split(" "):
        if w in ldict.keys():
            transformed.append(ldict[w])
        else:
            transformed.append(ldict['<ID>'])
    return transformed

lines = [ sentence_to_num(s, reduced_w2n_dict) for s in cleaned_text.split("<EOL>") ]

print(lines[20:30])

[[161, 161, 161, 161, 161, 161, 119, 161, 161, 107, 161, 107, 161, 161, 91, 161, 161, 107, 161, 107, 161, 107, 161, 161, 72, 15, 151, 161, 107, 161, 72, 133, 151, 161, 107, 161, 72, 133, 151, 109, 161, 107, 161, 107, 153, 107, 161, 161, 107, 161, 107, 161, 161], [161, 161, 161], [161, 161, 73, 161, 73, 161, 161, 161, 25, 161, 161, 161, 73, 161, 73, 161, 161], [161, 161, 161], [161, 161, 161, 161, 161, 161, 119, 161, 161, 107, 161, 107, 161, 161, 91, 161, 161, 107, 161, 107, 161, 107, 161, 161, 72, 15, 151, 161, 107, 161, 72, 133, 151, 161, 107, 161, 72, 133, 151, 109, 161, 107, 161, 107, 153, 107, 161, 161, 107, 161, 107, 161, 161, 107, 161, 107, 161, 107, 161, 161], [161, 161, 161], [161, 161, 73, 161, 73, 161, 161, 105, 161, 161, 57, 161, 107, 161, 161, 73, 161, 73, 161, 161], [161, 161, 161], [161, 161, 161, 161, 161, 161, 119, 161, 161, 107, 70, 161, 91, 161, 161, 107, 132, 161, 72, 15, 151, 161, 107, 161, 72, 133, 151, 161, 107, 161, 72, 133, 151, 109, 161, 107, 197, 161, 107, 161

So far we got all sentences tokenized and words that are irrelevant marked as such. Now we can start turning these into input target pairs.

In [15]:
def sent_to_samples(sentence, window_size=1):
    
    # ls stands for labeled sentence (meaning that the
    # end of the sentence is marked with an <EOL>)
    ls = sentence + [ reduced_w2n_dict['<EOL>'] ]
    
    x_vals = [ ls[i:i+window_size] for i in range(len(ls) - window_size) ]
    y_vals = [ ls[i+window_size] for i in range(len(ls) - window_size) ]
        
    return x_vals, y_vals

In [16]:
inputs = []
labels = []

for line in lines:
    x, y = sent_to_samples(line)
    inputs += x
    labels += y

In [17]:
print(inputs[20:30])
print(labels[20:30])

[[25], [153], [204], [22], [145], [161], [11], [161], [161], [57]]
[153, 204, 22, 145, 161, 11, 161, 161, 57, 161]


In [18]:
# If you run out of memory, set this to a value between 0 and 1 to reduce the number of samples
keep_prob = 1.0

import random

# If the training data has been assigned before, delete it to
# free memory. If we just reassign it, we end up with a lot of
# wasted space and we won't be able to store the data in RAM any more
try:
    del training_inputs
except:
    pass

try:
    del training_labels
except:
    pass
    
training_inputs = []
training_labels = []

for idx in range(len(inputs)):
    if random.random() < keep_prob:
        training_inputs.append(inputs[idx])
        training_labels.append(labels[idx])
        
print("Kept {} out of {} samples".format(len(training_inputs), len(inputs)) )

Kept 8082959 out of 8082959 samples


Let's put all this madness into a handy little function.

In [23]:
def make_training_data(cleaned_text, w2n, sampling_rate = 1.0, window_size=1):
    
    lines = [ sentence_to_num(s, w2n) for s in cleaned_text.split("<EOL>") ]
    
    inputs = []
    labels = []

    for line in lines:
        x, y = sent_to_samples(line, window_size=window_size)
        inputs += x
        labels += y
        
    training_inputs = []
    training_labels = []

    for idx in range(len(inputs)):
        if random.random() < sampling_rate:
            training_inputs.append(inputs[idx])
            training_labels.append(labels[idx])
            
    training_inputs = np.array(training_inputs)
    training_labels = to_categorical(training_labels)
    
    return training_inputs, training_labels

### Time to introduce the neural network

Now that we got our data set up, we can cast it into a numpy array and one-hot-encode the labels. This will take about 30G of RAM.

In [20]:
import numpy as np
from keras.utils import to_categorical

training_inputs = np.array(training_inputs)
training_labels = to_categorical(training_labels)

Using TensorFlow backend.


Now we can set some parameters for our network that will influence its performance. Some good default values have been already chosen for you.

In [21]:
learning_rate = 1e-4
embedding_dim = 128
mem_size = 128 # Number of recurrent cells (GRU in our case)

epochs = 2
batch_size = 256
validation_split = 0.1

# ----
load_pretrained = True
pretrained_date = "2018-02-20--22-21-59"

# --- Auto computed
vocab_size = len(reduced_w2n_dict)

The code below is pretty much boilerplate for our case, so there is no need to change it.

In [24]:
from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding
from keras.optimizers import Adam

if load_pretrained:
    print("Loading pretrained model instead of starting over")
    from keras.models import load_model
    
    model_file = pretrained_date + "--model.h5"
    w2n_file = pretrained_date + "--w2n.dict"
    n2w_file = pretrained_date + "--n2w.dict"
    
    print("File: " + model_file)
    model = load_model(model_file)
    
    model.summary()
    
    print("Reshaping the training data...")
    
    import pickle
    
    with open(w2n_file, 'rb') as f:
        reduced_w2n_dict = pickle.load(f)
            
    with open(n2w_file, 'rb') as f:
        reduced_n2w_dict = pickle.load(f)

    try:
        del training_inputs
    except:
        pass
    
    try:
        del training_labels
    except:
        pass
    
    training_inputs, training_labels = make_training_data(cleaned_text, reduced_w2n_dict)
    
    print("Done")
    
else:
    model = Sequential()

    model.add( Embedding(vocab_size, embedding_dim, input_length=1) )
    model.add( GRU(mem_size) )
    model.add( Dense(vocab_size, activation='softmax') )
    
    model.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy')

    model.summary()

    model.fit(x = training_inputs, y = training_labels, epochs = epochs, \
          batch_size = batch_size, verbose = 1, validation_split = validation_split)

Loading pretrained model instead of starting over
File: 2018-02-20--22-21-59--model.h5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1, 128)            27392     
_________________________________________________________________
gru_2 (GRU)                  (None, 128)               98688     
_________________________________________________________________
dense_2 (Dense)              (None, 214)               27606     
Total params: 153,686
Trainable params: 153,686
Non-trainable params: 0
_________________________________________________________________
Reshaping the training data...
Done


### Testing the network

Let's throw some words at the network and see what it predicts.

In [27]:
test_words = ['if', 'import', 'def', '(']
top_n = 5

for word in test_words:
    test_int = reduced_w2n_dict[word]
    
    output = model.predict(np.array([test_int]))
    
    candidates = output.argsort()[0][::-1]
    
    for idx in range(top_n):
        print(word, "->", reduced_n2w_dict[candidates[idx]])

if -> <ID>
if -> not
if -> self
if -> __name__
if -> len
import -> <ID>
import -> sys
import -> os
import -> time
import -> random
def -> <ID>
def -> __init__
def -> main
def -> get
def -> close
( -> <ID>
( -> self
( -> 0
( -> 1
( -> x


Looking good!

Now we should save the model to reuse it later.

In [26]:
if not load_pretrained:
    import time
    import pickle

    timestr = time.strftime("%Y-%m-%d--%H-%M-%S")

    filename = timestr + "--model.h5"
    model.save(filename)

    with open(timestr + "--w2n.dict", 'wb') as f:
        pickle.dump(reduced_w2n_dict, f)
    
    with open(timestr + "--n2w.dict", 'wb') as f:
        pickle.dump(reduced_n2w_dict, f)
    
    print("Saved with timestamp " + timestr)
else:
    print("Was already loaded from a file, so not saving.")

Was already loaded from a file, so not saving.


# Part 2 - Making longer predictions

We are honestly a little bit surprised how well the first test turned out. The goal in this section is now slightly more advanced:

- Train the network on bigrams to let it learn about context
- Make predictions about whole lines, that is, until an ```<EOL>``` is predicted
- Reuse as much from the previous part as possible

So let's get straight to it!

In [28]:
try:
    del training_inputs
except:
    pass
    
try:
    del training_labels
except:
    pass

In [29]:
window_size = 2

In [30]:
training_inputs, training_labels = make_training_data(cleaned_text, reduced_w2n_dict, window_size = window_size)

In [31]:
print(training_inputs[20:30])

[[154 176]
 [176 176]
 [176   1]
 [  1 176]
 [176 184]
 [184 176]
 [176 204]
 [204 134]
 [134 176]
 [176  90]]


In [32]:
model_ngram = Sequential()

model_ngram.add( Embedding(vocab_size, embedding_dim, input_length=window_size) )
model_ngram.add( GRU(mem_size) )
model_ngram.add( Dense(vocab_size, activation='softmax') )

model_ngram.compile(optimizer=Adam(learning_rate), loss='categorical_crossentropy')

In [33]:
model_ngram.fit(x=training_inputs, y=training_labels, epochs=epochs, \
                batch_size=batch_size, validation_split=validation_split, verbose=1)

Train on 6808690 samples, validate on 756522 samples
Epoch 1/2
6808690/6808690 [==============================] - 241s 35us/step - loss: 1.4798 - val_loss: 1.4136
Epoch 2/2
6808690/6808690 [==============================] - 239s 35us/step - loss: 1.3403 - val_loss: 1.3864


In [34]:
test_words = [ ['if', '__name__'], ['def', 'main'], ['main', '('] ]
top_n = 4

for word in test_words:
    test_input = [ [reduced_w2n_dict[word[0]], reduced_w2n_dict[word[0]]] ]
    
    output = model_ngram.predict(np.array(test_input))
    
    candidates = output.argsort()[0][::-1]
    
    for idx in range(top_n):
        print(word, "->", reduced_n2w_dict[candidates[idx]])

['if', '__name__'] -> <ID>
['if', '__name__'] -> isinstance
['if', '__name__'] -> self
['if', '__name__'] -> not
['def', 'main'] -> <ID>
['def', 'main'] -> (
['def', 'main'] -> __init__
['def', 'main'] -> main
['main', '('] -> (
['main', '('] -> <ID>
['main', '('] -> .
['main', '('] -> is


In [49]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file="model-single.png")

ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.